# Number Gesture Classification using CNN

### Importing Neccessary Libraries

In [1]:
import numpy as np#used for numerical analysis
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras import layers #A layer consists of a tensor-in tensor-out computation function
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D,MaxPooling2D #Convolutional layer
#MaxPooling2D-for downsampling the image
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


### Image Data Agumentation

In [23]:
#setting parameter for Image Data agumentation to the traing data
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

### Loading our data and performing data agumentation

In [25]:
#performing data agumentation to train data
x_train = train_datagen.flow_from_directory('data/train',target_size=(64, 64),batch_size=5,
                                            color_mode='grayscale',class_mode='categorical')
#performing data agumentation to test data
x_test = test_datagen.flow_from_directory('data/test',target_size=(64, 64),batch_size=5,
                                            color_mode='grayscale',class_mode='categorical') 

Found 600 images belonging to 6 classes.
Found 30 images belonging to 6 classes.


In [26]:
print(x_train.class_indices)#checking the number of classes

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5}


In [27]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 100, 1: 100, 2: 100, 3: 100, 4: 100, 5: 100})

### Creating the model

In [28]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 1), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=6, activation='softmax')) # softmax for more than 2



In [29]:
classifier.summary()#summary of our model

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 62, 62, 32)        320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               802944    
_________________________________________________________________
dense_7 (Dense)              (None, 6)                

### Compiling the model

In [30]:
# Compiling the CNN
# categorical_crossentropy for more than 2
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

## Fitting the model

In [31]:
classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=20, validation_data=x_test,validation_steps = len(x_test))# No of images in test set

Epoch 1/20
120/120 [==============================] - 6s 54ms/step - loss: 1.5406 - accuracy: 0.3667 - val_loss: 0.9710 - val_accuracy: 0.5667
Epoch 2/20
120/120 [==============================] - 6s 52ms/step - loss: 0.7372 - accuracy: 0.7133 - val_loss: 0.4782 - val_accuracy: 0.8000
Epoch 3/20
120/120 [==============================] - 5s 41ms/step - loss: 0.4904 - accuracy: 0.7950 - val_loss: 0.8232 - val_accuracy: 0.7000
Epoch 4/20
120/120 [==============================] - 5s 43ms/step - loss: 0.4080 - accuracy: 0.8367 - val_loss: 0.2943 - val_accuracy: 0.8667
Epoch 5/20
120/120 [==============================] - 5s 39ms/step - loss: 0.2905 - accuracy: 0.8850 - val_loss: 0.3520 - val_accuracy: 0.9000
Epoch 6/20
120/120 [==============================] - 6s 46ms/step - loss: 0.2370 - accuracy: 0.9133 - val_loss: 0.3324 - val_accuracy: 0.8667
Epoch 7/20
120/120 [==============================] - 6s 47ms/step - loss: 0.2347 - accuracy: 0.9217 - val_loss: 0.1927 - val_accuracy: 0.9667

### Saving our model

In [32]:
# Save the model
classifier.save('gesture.h5')

In [33]:
model_json = classifier.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)

### Predicting our results

In [62]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model("gesture.h5") #loading the model for testing

In [69]:
img = image.load_img(r"E:\PROJECTS\number-sign-recognition\data\test\1\1.jpg",grayscale=True,
                     target_size= (64,64))#loading of the image
x = image.img_to_array(img)#image to array
x = np.expand_dims(x,axis = 0)#changing the shape
pred = model.predict_classes(x)#predicting the classes
pred

array([1], dtype=int64)

In [71]:
index=['0','1','2','3','4','5']
result=str(index[pred[0]])
result

'1'